# LMNA Brodsky et al, Cardiomyopathy, dilated, 1A 


The data in this workbook were derived from Table 1 of [Brodsky GL, et al (2000) Lamin A/C gene mutation associated with dilated cardiomyopathy with variable skeletal muscle involvement. Circulation;101(5):473-6. PMID:10662742](https://pubmed.ncbi.nlm.nih.gov/10662742/).

Note that we will code the following two features as excluded in all individuals based on knowledge external to the above cited publication.

- Excluded: Progeroid facial appearance HP:0005328 (see OMIM)
- Excluded: Lipodystrophy HP:0009125 (see OMIM)

In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', None) # show entire column contents, important!
from IPython.display import display, HTML
from pyphetools.creation import *
from pyphetools.visualization import *
from pyphetools.validation import *
import pyphetools
print(f"Using pyphetools version {pyphetools.__version__}")

Using pyphetools version 0.9.37


In [2]:
PMID = "PMID:10662742"
title = "Lamin A/C gene mutation associated with dilated cardiomyopathy with variable skeletal muscle involvement"
cite = Citation(pmid=PMID, title=title)
parser = HpoParser(hpo_json_file="../hp.json")
hpo_cr = parser.get_hpo_concept_recognizer()
hpo_version = parser.get_version()
hpo_ontology = parser.get_ontology()
metadata = MetaData(created_by="ORCID:0000-0002-0736-9199", citation=cite)
metadata.default_versions_with_hpo(version=hpo_version)
print(f"HPO version {hpo_version}")

HPO version 2024-01-16


In [3]:
df = pd.read_excel("input/Brodsky2000.xlsx")
df.head()

,ID,II-1,II-5,III-1,III-3,III-4
0,"Age, y",22,30,4,16,18
1,Sex,M,M,F,M,F
2,NYHA class,2,3,1,1,1
3,CK-MM,N-3×,N,N,3×,4×
4,Arrhythmia,"NSVT, SVT","AF, VT",PAC,"SVT, PVC",SVT


In [4]:
dft = df.transpose()
dft.columns = dft.iloc[0]
dft.drop(dft.index[0], inplace=True)
dft['individual_id'] = dft.index  # Set the new column 'patient_id' to be identical to the contents of the index
dft.head() # check the transposed table

ID,"Age, y",Sex,NYHA class,CK-MM,Arrhythmia,ECG,"LVEDD, cm","FS, %","EF, %",Neuromuscular,Pattern,"Duration, y","LVEF, %",Outcome,individual_id
II-1,22,M,2,N-3×,"NSVT, SVT","AVB 2°–3°,LBBB 1°",5.9,12,30,"Weakness, slight calf hypertrophy",Mild LGMD,11,27,"DCM, HF, transplant",II-1
II-5,30,M,3,N,"AF, VT","AVB 2°–3°,LBBB 1°",6,24,48,N,N,2,16,"HF, SD",II-5
III-1,4,F,1,N,PAC,N,2.8,36,…,Minimal weakness,Aspecific,2,…,Stable,III-1
III-3,16,M,1,3×,"SVT, PVC",N,4.8,33,49,Slight weakness of limb-girdle,Mild LGMD,3,57,Stable,III-3
III-4,18,F,1,4×,SVT,LAH,4.5,37,63,"Slight rigidity of spine, elbow, Achilles tendon",Mild EDMD,3,63,Stable,III-4


In [5]:
ageMapper = AgeColumnMapper.by_year(column_name="Age, y")
#ageMapper.preview_column(dft["Age, y"])
sexMapper = SexColumnMapper(male_symbol="M", female_symbol="F", column_name="Sex")
#sexMapper.preview_column(dft["Sex"])

In [6]:
# used for creating mapper only
# result = OptionColumnMapper.autoformat(df=dft, concept_recognizer=hpo_cr)
column_mapper_list = list()

In [7]:
ck_mm_d = {'N-3×': 'Increased circulating creatine kinase MM isoform',
 '3×': 'Increased circulating creatine kinase MM isoform',
 '4×': 'Increased circulating creatine kinase MM isoform'}
ckmmMapper = OptionColumnMapper(column_name='CK-MM',concept_recognizer=hpo_cr, option_d=ck_mm_d)
column_mapper_list.append(ckmmMapper)
ckmmMapper.preview_column(dft)

,mapping,count
0,Increased circulating creatine kinase MM isoform (HP:0032234) (observed),3


In [8]:
#dft['NYHA class'].unique() # 1, 2, 3 -- all of which indicate heart failure
chfMapper = ConstantColumnMapper(column_name='NYHA class',hpo_id="HP:0001635", hpo_label="Congestive heart failure")
column_mapper_list.append(chfMapper)

In [9]:
arrhythmia_d = {'NSVT': 'Ventricular tachycardia',
 'SVT': 'Supraventricular tachycardia',
 'AF': 'Atrial fibrillation',
 'VT': 'Ventricular tachycardia',
 'PAC': 'Premature atrial contractions',
 'PVC': 'Premature ventricular contraction'}
arrhythmiaMapper = OptionColumnMapper(column_name='Arrhythmia',concept_recognizer=hpo_cr, option_d=arrhythmia_d)
column_mapper_list.append(arrhythmiaMapper)
arrhythmiaMapper.preview_column(dft)

,mapping,count
0,Ventricular tachycardia (HP:0004756) (observed),2
1,Supraventricular tachycardia (HP:0004755) (observed),3
2,Atrial fibrillation (HP:0005110) (observed),1
3,Premature atrial contractions (HP:0006699) (observed),1
4,Premature ventricular contraction (HP:0006682) (observed),1


In [10]:
ecg_d = {'AVB 2°–3°': 'Second degree atrioventricular block',
 'LBBB 1°': 'Left bundle branch block',
 'LAH': 'Left anterior fascicular block'}
excluded = {"N": "Cardiac conduction abnormality"}
ecgMapper = OptionColumnMapper(column_name='ECG',concept_recognizer=hpo_cr, option_d=ecg_d, excluded_d=excluded)
column_mapper_list.append(ecgMapper)
ecgMapper.preview_column(dft)

,mapping,count
0,Second degree atrioventricular block (HP:0011706) (observed),2
1,Left bundle branch block (HP:0011713) (observed),2
2,Cardiac conduction abnormality (HP:0031546) (excluded),2
3,Left anterior fascicular block (HP:0011711) (observed),1


In [11]:
# Normal values not provided in article. We will assume LVEDD. mm. F. 38-52. M. 42-58.
lvedd_d = {'5.9': 'Elevated left ventricular end-diastolic diameter',
}
excluded = { '2.8': 'Elevated left ventricular end-diastolic diameter',
 '4.8': 'Elevated left ventricular end-diastolic diameter',
 '4.5': 'Elevated left ventricular end-diastolic diameter'} 
lveddMapper = OptionColumnMapper(column_name='LVEDD, cm',concept_recognizer=hpo_cr, option_d=lvedd_d, excluded_d=excluded)
column_mapper_list.append(lveddMapper)
lveddMapper.preview_column(dft)

,mapping,count
0,Elevated left ventricular end-diastolic diameter (HP:0034307) (observed),1
1,Elevated left ventricular end-diastolic diameter (HP:0034307) (excluded),3


In [12]:
ef_d = {'30': 'Moderately reduced left ventricular ejection fraction',
 '48': 'Mildly reduced left ventricular ejection fraction',
 '49': 'Mildly reduced left ventricular ejection fraction',
 }
excluded = {'63': 'Reduced left ventricular ejection fraction'} 
efMapper = OptionColumnMapper(column_name='EF, %',concept_recognizer=hpo_cr, option_d=ef_d, excluded_d=excluded)
column_mapper_list.append(efMapper)
efMapper.preview_column(dft)

,mapping,count
0,Moderately reduced left ventricular ejection fraction (HP:0012665) (observed),1
1,Mildly reduced left ventricular ejection fraction (HP:0012663) (observed),2
2,Reduced left ventricular ejection fraction (HP:0012664) (excluded),1


In [13]:
neuromuscular_d = {'Weakness': 'Muscle weakness',
 'slight calf hypertrophy': 'Calf muscle hypertrophy',
 'Minimal weakness': 'Asthenia',
 'Slight weakness of limb-girdle': 'Asthenia',
 'Slight rigidity of spine': 'Spinal rigidity',}
neuromuscularMapper = OptionColumnMapper(column_name='Neuromuscular',concept_recognizer=hpo_cr, option_d=neuromuscular_d)
column_mapper_list.append(neuromuscularMapper)
neuromuscularMapper.preview_column(dft)

,mapping,count
0,Muscle weakness (HP:0001324) (observed),1
1,Calf muscle hypertrophy (HP:0008981) (observed),1
2,Asthenia (HP:0025406) (observed),2
3,Spinal rigidity (HP:0003306) (observed),1


In [14]:
outcome_d = {'DCM': 'Dilated cardiomyopathy',
 'HF': 'Congestive heart failure',
 'transplant': 'Congestive heart failure',
 'SD': 'Sudden cardiac death',
 }
outcomeMapper = OptionColumnMapper(column_name='Outcome',concept_recognizer=hpo_cr, option_d=outcome_d)
column_mapper_list.append(outcomeMapper)
outcomeMapper.preview_column(dft)

,mapping,count
0,Dilated cardiomyopathy (HP:0001644) (observed),1
1,Congestive heart failure (HP:0001635) (observed),3
2,Sudden cardiac death (HP:0001645) (observed),1


In [15]:
encoder = CohortEncoder(df=dft, 
                        hpo_cr=hpo_cr, 
                        column_mapper_list=column_mapper_list, 
                        individual_column_name="individual_id", 
                        agemapper=ageMapper, 
                        sexmapper=sexMapper,
                        metadata=metadata)
omim_id = "OMIM:115200"
omim_label = "Cardiomyopathy, dilated, 1A"
 	 
cmp1a = Disease(disease_id=omim_id, disease_label=omim_label)
encoder.set_disease(disease=cmp1a)

In [16]:
LMNA_transcript = "NM_005572.4"
vvalidator = VariantValidator(genome_build="hg38", transcript=LMNA_transcript)
# heterozygosity in affected members for a 1-bp deletion (del959T) deletion in exon 6 of the LMNA gene
# NM_170707.4(LMNA):c.960del (p.Arg321fs) 
var = vvalidator.encode_hgvs("c.960del")
var.set_heterozygous()

https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_005572.4%3Ac.960del/NM_005572.4?content-type=application%2Fjson


In [17]:
individuals = encoder.get_individuals()
for i in individuals:
    i.add_variant(var)
# We code the following terms as excluded (see top of notebook)
for i in individuals:
    progeroid = HpTerm(hpo_id="HP:0005328", label="Progeroid facial appearance", observed=False)
    lipodystrophy = HpTerm(hpo_id="HP:0009125", label="Lipodystrophy", observed=False)
    i.add_hpo_term(progeroid)
    i.add_hpo_term(lipodystrophy)    

In [18]:
cvalidator = CohortValidator(cohort=individuals, ontology=hpo_ontology, min_hpo=1, allelic_requirement=AllelicRequirement.MONO_ALLELIC)
qc = QcVisualizer(cohort_validator=cvalidator)
display(HTML(qc.to_summary_html()))

Level,Error category,Count


In [19]:
phenopackets = [i.to_ga4gh_phenopacket(metadata=metadata.to_ga4gh()) for i in individuals]
table = PhenopacketTable(phenopacket_list=phenopackets)
display(HTML(table.to_html()))

Individual,Disease,Genotype,Phenotypic features
II-1 (MALE; P22Y),"Cardiomyopathy, dilated, 1A (OMIM:115200)",NM_005572.4:c.960del (heterozygous),Second degree atrioventricular block (HP:0011706); Supraventricular tachycardia (HP:0004755); Increased circulating creatine kinase MM isoform (HP:0032234); Moderately reduced left ventricular ejection fraction (HP:0012665); Dilated cardiomyopathy (HP:0001644); Calf muscle hypertrophy (HP:0008981); Muscle weakness (HP:0001324); Elevated left ventricular end-diastolic diameter (HP:0034307); Left bundle branch block (HP:0011713); Congestive heart failure (HP:0001635); Ventricular tachycardia (HP:0004756); excluded: Progeroid facial appearance (HP:0005328); excluded: Lipodystrophy (HP:0009125)
II-5 (MALE; P30Y),"Cardiomyopathy, dilated, 1A (OMIM:115200)",NM_005572.4:c.960del (heterozygous),Second degree atrioventricular block (HP:0011706); Sudden cardiac death (HP:0001645); Atrial fibrillation (HP:0005110); Mildly reduced left ventricular ejection fraction (HP:0012663); Left bundle branch block (HP:0011713); Congestive heart failure (HP:0001635); Ventricular tachycardia (HP:0004756); excluded: Progeroid facial appearance (HP:0005328); excluded: Lipodystrophy (HP:0009125)
III-1 (FEMALE; P4Y),"Cardiomyopathy, dilated, 1A (OMIM:115200)",NM_005572.4:c.960del (heterozygous),Congestive heart failure (HP:0001635); Premature atrial contractions (HP:0006699); Asthenia (HP:0025406); excluded: Cardiac conduction abnormality (HP:0031546); excluded: Elevated left ventricular end-diastolic diameter (HP:0034307); excluded: Progeroid facial appearance (HP:0005328); excluded: Lipodystrophy (HP:0009125)
III-3 (MALE; P16Y),"Cardiomyopathy, dilated, 1A (OMIM:115200)",NM_005572.4:c.960del (heterozygous),Increased circulating creatine kinase MM isoform (HP:0032234); Congestive heart failure (HP:0001635); Supraventricular tachycardia (HP:0004755); Premature ventricular contraction (HP:0006682); Mildly reduced left ventricular ejection fraction (HP:0012663); Asthenia (HP:0025406); excluded: Cardiac conduction abnormality (HP:0031546); excluded: Elevated left ventricular end-diastolic diameter (HP:0034307); excluded: Progeroid facial appearance (HP:0005328); excluded: Lipodystrophy (HP:0009125)
III-4 (FEMALE; P18Y),"Cardiomyopathy, dilated, 1A (OMIM:115200)",NM_005572.4:c.960del (heterozygous),Increased circulating creatine kinase MM isoform (HP:0032234); Congestive heart failure (HP:0001635); Supraventricular tachycardia (HP:0004755); Left anterior fascicular block (HP:0011711); Spinal rigidity (HP:0003306); excluded: Elevated left ventricular end-diastolic diameter (HP:0034307); excluded: Reduced left ventricular ejection fraction (HP:0012664); excluded: Progeroid facial appearance (HP:0005328); excluded: Lipodystrophy (HP:0009125)


In [20]:
Individual.output_individuals_as_phenopackets(individual_list=individuals,
                                              metadata=metadata)

We output 5 GA4GH phenopackets to the directory phenopackets
